In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Distribution de la température dans un appartement d'un immeuble aux plusieurs étages


# Équation de transfert de chaleur:
# k*(d^2 T(x,y)/dx^2 + d^2 T(x,y)/dy^2)+S=0


# Conditions aux limites:

# (1) Condition convective (de Robin) à x=0 et à x=Lx (faces externes du mur):
# -k*dT(x=0,y)/dx=-h*(T-Ta)
# -k*dT(x=L,y)/dx=h*(T-Ta)
Ta=-20 #oC

# (2) Condition de Dirichlet sur le plafond et sur le plancher
# T(x, y=0 ou y=Ly)=Tp
Tp=20 #oC

# Dimensions d'appartement
Lx=4 #[m]
Ly=2.4; #[m]

# Parametres d'un mur d'isolation thermique
Lm=0.4 #m ; Épaisseur du mur en brique
km=0.85#W/(m*K); La conductivité thermique de la brique
h=1 #W/(m^2*K); Coefficient de transfert thermique sur les surfaces extérieures du mur

# Paramètres de l'air qui remplit l'appartement
ka=0.024

q=1.0e4;# W/m^3;

d_ar = 0.1*np.array([8], dtype=np.double)
tini_ar=np.zeros(d_ar.size,dtype=np.double)
tinv_ar=np.zeros(d_ar.size,dtype=np.double)
mem_ar=np.zeros(d_ar.size,dtype=np.double)
Tm_ar=np.zeros(d_ar.size,dtype=np.double)
Err_ar=np.zeros(d_ar.size-1,dtype=np.double)
d_Err_ar=np.zeros(d_ar.size-1,dtype=np.double)


def Mindex(i,j): #Associé la case i,j à sa colone dans la matrice M
    index=(j-1)*Ny+i
    return index-1

for d in d_ar:
    Nx=int(np.rint(Lx/d+1)) # Nombre de nœuds le long de X
    Ny=int(np.rint(Ly/d+1)) # Nombre de nœuds le long de Y

    iniTimeBefore=time.time_ns()
    S=np.zeros((Ny,Nx),dtype=np.double)
    k=np.zeros((Ny,Nx),dtype=np.double)
    
    #Définition de la source et des 
    for i in np.arange(1,Ny+1,1): #i=1,..,Ny - numérotation des nœuds sur un maillage physique
        y=(i-1)*d
        for j in np.arange(1,Nx+1,1): #j=1,..,Nx - numérotation des nœuds sur un maillage physique
            x=(j-1)*d
            
            # Sourse volumique de chaleur q[W/m^3] d'épaisseur dL.
            # La source est intégrée dans les parties intérieures du mur à x=Lm et à x=Lx-Lm et
            # il occupe les tiers du mur dans la direction verticale
            dL=0.1

            if ((x<=Lm) and (y<=Ly/3+Lm) and (y>Lm)):
                # À l'intérieur de l'élément chauffant
                S[i-1,j-1]=q*np.exp(-((x-Lm)/dL)**2)
            elif ((x>=(Lx-Lm)) and (y<=Ly/3+Lm) and (y>Lm)):
                # À l'intérieur de l'élément chauffant
                S[i-1,j-1]=q*np.exp(-((Lx-Lm-x)/dL)**2)
            else:
                # À l'extérieur de l'élément chauffant
                S[i-1,j-1]=0.0
            
            # L'espace de vie de l'appartement est délimité par
            # les parois d'épaisseur Lm à tous les quatre côtés
            if ((x<=Lm) or (x>=(Lx-Lm)) or (y<=Lm) or (y>=(Ly-Lm))):
                # À l'intérieur du mur
                k[i-1,j-1]=km
            else:
                # À l'intérieurde de l'appartement
                k[i-1,j-1]=ka
    M=np.zeros((Nx*Ny,Nx*Ny),dtype=np.double);
    b=np.zeros((Nx*Ny,1),dtype=np.double);
    T=np.zeros((Nx*Ny,1),dtype=np.double);
    Tr=np.zeros((Ny,Nx),dtype=np.double);

    for i in np.arange(1, Ny+1,1):
        for j in np.arange(1, Nx+1,1):
            pl = i+(j-1)*Ny

            if (((i>1) and (i<Ny)) and ((j>1) and (j<Nx))):
                # noeud qui est strictement à l'intérieur de la cellule de simulation
                pc=pl
                M[pl-1,pc-1]=-4 # contribution de noeud (i,j)
                pc=(i-1)*Nx+j-1
                M[pl-1,pc-1]=1 # contribution de noeud (i,j-1)
                pc=(i-1)*Nx+j+1
                M[pl-1,pc-1]=1 # contribution de noeud (i,j+1)
                pc=(i-2)*Nx+j
                M[pl-1,pc-1]=1 # contribution de noeud (i-1,j)
                pc=(i)*Nx+j
                M[pl-1,pc-1]=1 # contribution de noeud (i+1,j)
                b[pl-1]=-d**2*S[i-1,j-1]/k[i-1,j-1]
            elif j==1:
                M[Mindex(i,j),Mindex(i,j)] = 3+2*d*h/k[i-1,j-1]
                M[Mindex(i,j), Mindex(i,j+1)] = -4
                M[Mindex(i,j), Mindex(i,j+2)] = 1
            
            elif j==Nx:
                M[Mindex(i,j), Mindex(i,j)] = 3+2*d*h/k[i-1,j-1]
                M[Mindex(i,j), Mindex(i,j-1)] = -4
                M[Mindex(i,j), Mindex(i,j-2)] = 1
            elif i==1:
                M[Mindex(i,j),Mindex(i,j)] = 3+2*d*h/k[i-1,j-1]
                M[Mindex(i,j), Mindex(i+1,j)] = -4
                M[Mindex(i,j), Mindex(i+2,j)] = 1
            elif i==Ny:
                M[Mindex(i,j),Mindex(i,j)] = 3+2*d*h/k[i-1,j-1]
                M[Mindex(i,j), Mindex(i-1,j)] = -4
                M[Mindex(i,j), Mindex(i-2,j)] = 1

            